- Equations

equations: $\alpha-1+(\frac{\kappa_0}{\kappa_C,})^{\gamma-2}\\\
P_0- (\gamma-1)(\alpha\kappa_0)^{(\gamma-1)}\Gamma(1-\gamma,\alpha\kappa_0)\\\
k_{max_obs}-\alpha\kappa_C\\\
\bar{k}\alpha^2-(1-P_0)k_{bar_obs}\\\
\kappa_0(\gamma-1)-\bar{k}(\gamma-2))\\\
$

In [8]:
from scipy.optimize import fsolve
import mpmath


def solve(gamma,k_bar_obs,k_max_obs) :
    def equations(p):
        alpha, kappa0,kappaC,P0,k_bar = p
        return (alpha-1+(kappa0/kappaC)**(gamma-2),
            P0- (gamma-1)*(alpha*kappa0)**(gamma-1)*mpmath.gammainc(1-gamma,alpha*kappa0),
            k_max_obs-alpha*kappaC,
            k_bar*alpha**2-(1-P0)*k_bar_obs,
            kappa0*(gamma-1)-k_bar*(gamma-2))
    return  fsolve(equations, (1, 1,k_max_obs,0.5,k_bar_obs))

def verify(alpha, kappa0,kappaC,P0,k_bar,gamma,k_bar_obs,k_max_obs) :
    def equations(p):
        alpha, kappa0,kappaC,P0,k_bar = p
        return (alpha-1+(kappa0/kappaC)**(gamma-2),
            P0- (gamma-1)*(alpha*kappa0)**(gamma-1)*mpmath.gammainc(1-gamma,alpha*kappa0),
            k_max_obs-alpha*kappaC,
            k_bar*alpha**2-(1-P0)*k_bar_obs,
            kappa0*(gamma-1)-k_bar*(gamma-2))
    print(equations((alpha, kappa0,kappaC,P0,k_bar)))

In [7]:
from scipy.optimize import fsolve
import mpmath
def eq(t):
    x,y=t
    return x**3-19,y+2
fsolve(eq,(-1,2))

array([ 2.66840165, -2.        ])

- exponent

$1+\frac{\text{节点数}}{\sum \ln(\text{每个节点的度数})}$

In [9]:
import powerlaw
import numpy as np
import matplotlib.pyplot as plt

def get_exponent(graphe):
    return 1+(graphe.number_of_nodes()/sum(np.log(graphe.degree().values())))

def plot_gamma(graphe) :
    #fitness = powerlaw.Fit(graphe.degree().values(),xmin =1)
    #print fitness.alpha,fitness.xmin
    alpha = 1+(graphe.number_of_nodes()/sum(np.log(graphe.degree().values())))

    #powerlaw.plot_pdf(graphe.degree().values())
    xdata,ydata = powerlaw.pdf(graphe.degree().values())

    vala ={}
    xvala=[]
    yvala=[]
    somme = 0
    exceptions =[]
    last_exception = -1
    for value in graphe.degree().values() :
        somme+=1
        try :
            vala[value]+=1
        except :
            vala[value] =1

    for degree in range(1,max(vala.keys())+1) :
        try :
            yvala.append(float(vala[degree])/somme)
            xvala.append(degree)

        except :
            if len(exceptions) ==0 :
                last_exception =degree
                #print degree
            exceptions.append(degree)
    #slope, intercept, r_value, p_value, std_err = stats.linregress(
        #np.log(xvala[:last_exception]),np.log(yvala[:last_exception]))

    #print slope

    x = xvala

    plt.plot(x,np.power(x,-alpha),color = "orange")
    #plt.plot(x,np.power(x,-2.34296138661),color ="blue")
    #plfit.plfit(graphe.degree().values(),quiet = False,verbose = True,silent = False)
    plt.plot(xvala,yvala,color = "black")
    plt.yscale('log')
    plt.xscale('log')
    #plt.ylim([0,0.4])
    #plt.xlim([0,20])
    plt.show()
    #xdata,ydata=powerlaw.pdf(graphe.degree().values())

- 生成随机图
    - $\mu=\beta \times \frac{\sin(\frac{\pi}{\beta})}{2\pi\bar{k}}$；
    - $\kappa $为$PowerLaw(x_\text{min}=\kappa_0,\gamma)$生成的N个点；
    - $\theta$是在$(0,2\pi)$均匀分布的$N$个点；
    - th: N个$\theta$的array；
    - r 代表半径，i,j\j,i只计算一次。保存在r2中；
    - $khi=N\times d_{th}/(2\pi\times\mu\kappa\kappa_2)$
    
    
    

In [10]:
import math
import powerlaw
import numpy as np
import networkx as nx

In [1]:
def drange(start, stop, step): # 连续抛出几个数
    r = start
    while r < stop:
        yield r
        r += step

In [2]:
def clust(N, k_bar, kappa0, gamma, start=1.1, stop=10, step=0.1, nb_networks=100):
    def mapping_function(beta):
        mean, std = clust_beta_given(beta, N, k_bar, kappa0, gamma, nb_networks)
        return [beta, mean, std]

    return map(mapping_function, drange(start, stop, step))

In [3]:
def clust_beta_given(beta, N, k_bar, kappa0, gamma, nb_networks):
    def mapping_function(_):
        return get_clustering_from_random_network(beta, N, k_bar, kappa0, gamma)

    result = map(mapping_function, range(nb_networks))
    return np.mean(result), np.std(result)

In [5]:
def approximated_beta(N, k_bar, kappa0, gamma, c_real, start=1.01, step=0.1, nb_networks=100):
    path_followed = []

    def rec_approximated_beta(beta_min, beta, beta_max):
        if beta_max is not None and (beta_max - beta_min) < step:
            return beta
        c_mean, c_std = clust_beta_given(beta, N, k_bar, kappa0, gamma, nb_networks)
        path_followed.append([beta, c_mean, c_std])
        print(beta, c_mean, c_std)
        if c_mean > c_real:
            return rec_approximated_beta(beta_min, (beta + beta_min) / 2, beta)
        if beta_max is not None:
            return rec_approximated_beta(beta, (beta + beta_max) / 2, beta_max)
        return rec_approximated_beta(beta, 2 * beta, None)

    beta_optimal = rec_approximated_beta(start, 1 / (1 - c_real), None)
    return beta_optimal, path_followed

In [6]:
def get_clustering_from_random_network(beta, N, k_bar, kappa0, gamma):
    """
    :rtype: float
    """
    graphe = generate_random_network(beta, N, k_bar, kappa0, gamma)# 下方的随机图
    if all(v == 0 for v in nx.clustering(graphe).values()):#聚集系数的值
        return 0
    return nx.average_clustering(graphe, count_zeros=False)

In [7]:
def generate_random_network(beta, N, k_bar, kappa0, gamma):
    mu = beta * math.sin(math.pi / beta) / (2 * math.pi * k_bar)
    graphe_random = nx.Graph()
    kappa = powerlaw.Power_Law(xmin=kappa0, parameters=[gamma]).generate_random(N)# 按照参数生成一个大小为N的array
    theta = np.random.uniform(0, 2 * math.pi, N)
    # print 1+(len(kappa)/sum(np.log(kappa))) #1.6
    th = np.outer(np.ones(N), theta)#生成N个theta
    # print "th"
    th2 = np.outer(theta, np.ones(N))
    # print "th2"
    kp = np.outer(np.ones(N), kappa)
    # print "kp"
    kp2 = np.outer(kappa, np.ones(N))
    # print "kp2"
    dth = math.pi - np.abs(math.pi - np.abs(th - th2))#角距离
    # print "dth"
    khi = N * dth / (2 * math.pi * mu * kp * kp2)
    # print "khi"
    r = np.random.uniform(0, 1, [N, N])
    # print "r"
    r2 = np.triu(np.log(1 / r - 1) / beta, k=1)#提取不含对角线的上三角矩阵
    p2 = np.triu(np.log(khi), k=1)
    # print np.where(r2>p2) ie ln(1/r-1) / beta > ln(khi) <=> r < 1/(1+khi^beta)
    graphe_random.add_edges_from(np.transpose(np.where(r2 > p2)))
    return graphe_random
    # print graphe_random.edges()
    # print graphe_random.number_of_nodes()
    # print graphe_random.number_of_edges()
    # print exponent.get_exponent(graphe_random)
    # print graphe_random.degree().values()
    # print np.mean(graphe_random.degree().values())
    # print max(graphe_random.degree().values())

- 似然函数

In [12]:
# coding=utf-8
import networkx as nx
import math
import numpy as np
import operator


def optimisation_likelihood(graphe, constantes, theta_init=None, subgroup_of_nodes=None):
    """On maximise la likelihood, en la maximisant pour chaque i separement
    theta_init est la configuration initiale de l'optimisation
    subgroup_of_nodes est le sous groupe pour lesquels on maximise la likelihood, les autres restent inchanges"""
    kappa0, gamma, beta, Nobs, k_bar = constantes

    kappa = {node: max(kappa0, degree - gamma / beta) for node, degree in graphe.degree().items()}
    mu = beta * math.sin(math.pi / beta) / (2 * math.pi * k_bar)
    a = (nx.to_numpy_matrix(graphe)).A

    if theta_init is None:
        theta_current = 2 * math.pi * np.random.randint(0, Nobs, Nobs) / Nobs
    else:
        theta_current = theta_init

    def localloglikelihood(theta_angle, node):
        """maximisation de la likelihood sur node"""
        th2 = theta_current
        dth = math.pi - np.abs(math.pi - np.abs(theta_angle - th2))
        khi = Nobs * dth / 2 * math.pi * mu * kappa[node] * kappa.values()
        p = 1 / (1 + khi ** beta)

        lp = np.log(p)
        lp[a[node] == 0] = 0
        l_p = np.log(1 - p)
        l_p[a[node] == 1] = 0
        alp = lp + l_p
        return np.sum(alp)

    theta_test = np.linspace(0, 2 * math.pi, num=Nobs)

    def maximize_locally(node):
        """"""
        lll_vector = map(lambda theta_angle: localloglikelihood(theta_angle, node), theta_test)
        theta_optimal = theta_test[np.argmax(lll_vector)]
        return theta_optimal

    if subgroup_of_nodes is None: subgroup_of_nodes = graphe.nodes()

    best_theta = theta_init
    best_likelihood = verifyloglikelihood(theta_init, graphe, constantes, silent=False)
    nb_retours = 0
    while (nb_retours < 2):
        for node, degree in sorted(graphe.degree_iter(subgroup_of_nodes), key=operator.itemgetter(1), reverse=True):
            theta_new = maximize_locally(node)
            if degree == 2:
                neighbor, neighbor_degree = max(graphe.degree_iter(graphe.neighbors(node)), key=operator.itemgetter(1))
                theta_new2 = theta_current[neighbor]
                print(2, theta_new, theta_new2)
            if degree == 1:
                theta_new1 = theta_current[graphe.neighbors(node)[0]]
                print(1, theta_new, theta_new1)

            theta_current[node] = theta_new
        current_likelihood = verifyloglikelihood(theta_current, graphe, constantes, silent=False)
        if current_likelihood < best_likelihood:
            nb_retours += 1
        else:
            best_likelihood = current_likelihood
            best_theta = theta_current
            nb_retours = 0
    return best_theta

'''
def optimisation_complete_par_etapes_likelihood(graphe, bornes, theta0, constantes):
    """On maximise la likelihood en commencant par des sous graphes contenant les noeuds principaux,
    on optimise sur le sous graphe complet à chaque étape"""
    kappa0, gamma, beta, Nobs, k_bar = constantes

    def optimisation_etape_likelihood(nodes_subgraph, theta_global):
        """On maximise la likelihood sur le sous graphe"""
        mapping_old_new = {}
        mapping_new_old = {}
        for i in range(len(nodes_subgraph)):
            old = nodes_subgraph[i]
            mapping_old_new[old] = i
            mapping_new_old[i] = old
        graphe_intermediaire = graphe.subgraph(nodes_subgraph)
        graphe_intermediaire = nx.relabel_nodes(graphe_intermediaire, mapping_old_new)
        Nlocal = len(nodes_subgraph)
        local_variables = kappa0, gamma, beta, Nlocal, k_bar

        theta_init = np.asarray([theta_global[mapping_new_old[node]] for node in graphe_intermediaire])
        theta_local_opt = optimisation_likelihood(graphe_intermediaire, local_variables, theta_init=theta_init)

        theta_global_opt = theta_global

        for node in graphe_intermediaire:
            theta_global_opt[mapping_new_old[node]] = theta_local_opt[node]
        return theta_global_opt

    theta_current = theta0
    for borne in bornes:
        nodes_subgraph = [node for node, degree in
                          sorted(graphe.degree_iter(), key=operator.itemgetter(1), reverse=True) if
                          degree >= borne]

        theta_current = optimisation_etape_likelihood(nodes_subgraph, theta_current)

    return theta_current
'''

In [ ]:
def subgraph_mapping(graphe, nodes_subgraph):
    mapping_old_new = {}
    mapping_new_old = {}
    for i in range(len(nodes_subgraph)):
        old = nodes_subgraph[i]
        mapping_old_new[old] = i
        mapping_new_old[i] = old
        # new network
    graphe_intermediaire = graphe.subgraph(nodes_subgraph)
    graphe_intermediaire = nx.relabel_nodes(graphe_intermediaire, mapping_old_new)
    return graphe_intermediaire, mapping_old_new, mapping_new_old

In [ ]:
def optimisation_par_etapes_likelihood(graphe, bornes, borne_lim, theta0, constantes, method="degree"):
    """On maximise la likelihood en commencant par des sous graphes contenant les noeuds principaux,
    on optimise uniquement sur els nouveaux noeuds à chaque étape"""
    kappa0, gamma, beta, Nobs, k_bar = constantes

    def optimisation_etape_likelihood(nodes_subgraph, nodes_evaluated, theta_global):
        # rename nodes of new network
        graphe_intermediaire, mapping_old_new, mapping_new_old = subgraph_mapping(graphe, nodes_subgraph)
        # new "constants"
        nodes_evaluated_local = map(lambda node: mapping_old_new[node], nodes_evaluated)
        Nlocal = len(nodes_subgraph)
        local_variables = kappa0, gamma, beta, Nlocal, k_bar
        # optimize on subgraph
        theta_init = np.asarray([theta_global[mapping_new_old[node]] for node in graphe_intermediaire])
        theta_local_opt = optimisation_likelihood(graphe_intermediaire, local_variables,
                                                  theta_init=theta_init,
                                                  subgroup_of_nodes=nodes_evaluated_local)
        # update theta values
        theta_global_opt = theta_global

        for node in graphe_intermediaire:
            theta_global_opt[mapping_new_old[node]] = theta_local_opt[node]
        return theta_global_opt
    '''
    if method == "degree":
        theta_current = theta0
        borne_sup = float("inf")
        degree_dict = graphe.degree_iter()
        for borne_inf in bornes:
            nodes_subgraph = [node for node, degree in degree_dict if degree >= borne_inf]
            if borne_inf > borne_lim:
                """on optimise sur tous les noeuds du sous graphe"""
                theta_current = optimisation_etape_likelihood(nodes_subgraph, nodes_subgraph, theta_current)
            else:
                """on optimise uniquement sur les noeuds qui viennent juste d'etre ajoutés"""
                nodes_evaluated = [node for node, degree in degree_dict if borne_sup > degree >= borne_inf]
                theta_current = optimisation_etape_likelihood(nodes_subgraph, nodes_evaluated, theta_current)
            borne_sup = borne_inf
    '''
    if method == "core_number":
        theta_current = theta0

        core_dict = nx.core_number(graphe)
        bornes = sorted(set(core_dict.values()), reverse=True)
        borne_sup = float("inf")
        theta_precedent_centre = [theta_current[node] for node, core_num in core_dict.iteritems() if core_num == 27]
        theta_courant_centre = [theta_current[node] for node, core_num in core_dict.iteritems() if core_num == 27]
        for borne_inf in bornes:
            nodes_subgraph = [node for node, core_number in core_dict.iteritems() if core_number >= borne_inf]
            if borne_inf > borne_lim:
                """on optimise sur tous les noeuds du sous graphe"""
                theta_current = optimisation_etape_likelihood(nodes_subgraph, nodes_subgraph, theta_current)
                theta_courant_centre = [theta_current[node] for node, core_num in core_dict.iteritems() if
                                        core_num == 27]
                print(borne_inf, np.mean(np.abs(
                    math.pi - np.absolute(np.asarray(theta_courant_centre) - np.asarray(theta_precedent_centre)))), \
                    np.max(np.abs(
                        math.pi - np.absolute(np.asarray(theta_courant_centre) - np.asarray(theta_precedent_centre)))))
                theta_precedent_centre = theta_courant_centre
            else:
                """on optimise uniquement sur les noeuds qui viennent juste d'etre ajoutés"""
                nodes_evaluated = [node for node, core_number in core_dict.iteritems() if
                                   borne_sup > core_number >= borne_inf]
                theta_current = optimisation_etape_likelihood(nodes_subgraph, nodes_evaluated, theta_current)
            borne_sup = borne_inf

    return theta_current
    """On maximise la likelihood en commencant par des sous graphes contenant les noeuds principaux,
    on optimise sur le sous graphe complet à chaque étape"""

In [ ]:
def verifyloglikelihood(theta_vector, graphe, constantes, silent=False):
    """On calcule somme (aij * logpij + (1-aij)* log(1-pij))"""
    # aij
    a = (nx.to_numpy_matrix(graphe)).A

    # pij
    kappa0, gamma, beta, Nobs, k_bar = constantes
    mu = beta * math.sin(math.pi / beta) / (2 * math.pi * k_bar)
    kappa = {node: max(kappa0, degree - gamma / beta) for node, degree in graphe.degree().items()}
    kp = np.outer(np.ones(Nobs), kappa.values())
    kp2 = np.outer(kappa.values(), np.ones(Nobs))
    mukpkp2 = 2 * math.pi * mu * kp * kp2
    th = np.outer(np.ones(Nobs), theta_vector)
    th2 = np.outer(theta_vector, np.ones(Nobs))
    dth = math.pi - np.abs(math.pi - np.abs(th - th2))
    khi = Nobs * dth / mukpkp2
    p = 1 / (1 + khi ** beta)

    # log(pij)
    lp = np.triu(np.log(p), k=1)
    l_p = np.triu(np.log(1 - p), k=1)

    # aij*log(pij)
    lp[a == 0] = 0
    l_p[a == 1] = 0
    if not silent: print(np.sum(lp + l_p), np.sum(lp), np.sum(l_p))
    return np.sum(lp + l_p)

In [ ]:
def rayon(graphe, constantes):
    kappa0, gamma, beta, Nobs, k_bar = constantes
    kappa = np.asarray([max(kappa0, degree - gamma / beta) for node, degree in graphe.degree().items()])
    mu = beta * math.sin(math.pi / beta) / (2 * math.pi * k_bar)
    rayon = 2 * np.log(Nobs / (math.pi * mu * kappa * kappa0))
    return rayon